In [1]:
##################################################################################
##### Define all parameters for model tuning
##################################################################################

n_fold = 5
expName = "NT_Site_PredNTS_Classification_DLNN_AsimEmbedding_DDE"
outPath = "Results"
foldName = "folds.pickle"

shuffle = True
seed = None

input_data_folder = "Data_from_Asim"
file = "Protein_DDE[100, 0, 0, 0]-st-simplesequence.csv"

monitor = "val_accuracy"

In [2]:
import os 
import pickle
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, confusion_matrix
from sklearn.metrics import roc_auc_score, matthews_corrcoef

import math

In [3]:
# for root, dirs, files in os.walk(input_data_folder):
#     for file in files:
#         print(file)

In [4]:
# print(tf.test.is_gpu_available(cuda_only=True))
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
##################################################################################
##### Build k-fold functions
##################################################################################

## Build the K-fold from dataset
def build_kfold(features, labels, k=10, shuffle=False, seed=None):
    
    skf = StratifiedKFold(n_splits=k, shuffle=shuffle, random_state=seed)
    kfoldList = []
    for train_index, test_index in skf.split(features, labels):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        kfoldList.append({
            "X_train": X_train,
            "X_test": X_test,
            "y_train":y_train,
            "y_test":y_test
        })
    return kfoldList

In [6]:
##################################################################################
##### define evaluator functions
##################################################################################

def pred2label(y_pred):
    y_pred = np.round(y_pred).astype(int)
    return y_pred

# Prepare Training and Independent data

In [7]:
input_data_file = os.path.join(input_data_folder, file)

data = pd.read_csv(input_data_file, sep=',', header=0)

train_data = data[data['set'] == 'train'].drop('set', axis=1)
independent_data = data[data['set'] == 'test'].drop('set', axis=1)

train_labels = np.array(train_data['labels'])
train_labels = train_labels.reshape((train_labels.shape[0], 1))

train_features = np.array(train_data.drop('labels', axis=1))

indpe_labels = np.array(independent_data['labels'])
indpe_labels = indpe_labels.reshape((indpe_labels.shape[0], 1))

indpe_features = np.array(independent_data.drop('labels', axis=1))

##################################################################################
##### extract data from the current fasta file
##################################################################################

print("\n======================================================================")
print("\nFile:", file)
print("Training Positive:", np.sum(train_labels))
print("Training Negative:", train_labels.shape[0] - np.sum(train_labels))
print("Independent Positive:", np.sum(indpe_labels))
print("Independent Negative:", indpe_labels.shape[0] - np.sum(indpe_labels))
print("Feature size:", train_features[0].shape)

##################################################################################
##### Generate Folds from dataset, and store to file
##################################################################################

## Generate the k-fold dataset
folds = build_kfold(train_features, train_labels, k=n_fold, shuffle=shuffle, seed=seed)

## Write the k-fold dataset to file
foldPath = os.path.join(outPath, expName, "{}fold".format(n_fold))
if(not os.path.isdir(foldPath)):
    os.makedirs(foldPath)
pickle.dump(folds, open(os.path.join(foldPath, foldName), "wb"))



File: Protein_DDE[100, 0, 0, 0]-st-simplesequence.csv
Training Positive: 1191
Training Negative: 1191
Independent Positive: 203
Independent Negative: 1022
Feature size: (400,)


# Model

In [8]:
epochs = 100
batch_size = 16

##################################################################################
##### Function to customize the DLNN architecture with parameters
##################################################################################

def DLNN_Classifier(input_vec_shape,
                    dense_decode_units = 32, ## Dense layer parameters
                    prob = 0.5, learn_rate = 0.0005, loss = 'binary_crossentropy', metrics = 'accuracy'):
    
    beta = 0.001
    
    input1 = tf.keras.layers.Input(shape=input_vec_shape)
    
    ######################################################################################################
    ########  Classifier  ################################################################################
    ######################################################################################################
    
    y = tf.keras.layers.Dense(dense_decode_units, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta), 
                              activation = 'relu')(input1)
#     y = tf.keras.layers.GaussianNoise(stddev=0.1)(y)
    y = tf.keras.layers.Dropout(prob)(y)
    
#     y = tf.keras.layers.Dense(int(dense_decode_units/2), 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                               activation = 'relu')(y)
#     y = tf.keras.layers.GaussianNoise(stddev=0.1)(y)
#     y = tf.keras.layers.Dropout(prob)(y)
    
    y = tf.keras.layers.Dense(1, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta), 
                              activation = 'sigmoid')(y)

    ## Generate Model from input and output
    model = tf.keras.models.Model(inputs=input1, outputs=y)
    
    ## Compile model
    if(metrics != None):
        model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
                      loss = loss, metrics = metrics)
    else:
        model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
                      loss = loss)

    return model

In [9]:
# epochs = 100
# batch_size = 16

# ##################################################################################
# ##### Function to customize the DLNN architecture with parameters
# ##################################################################################

# def DLNN_Classifier(input_vec_shape,
#                     dense_decode_units = 32, ## Dense layer parameters
#                     prob = 0.5, learn_rate = 0.0005, loss = 'binary_crossentropy', metrics = 'accuracy'):
    
#     beta = 0.001
    
#     input1 = tf.keras.layers.Input(shape=input_vec_shape)
    
#     ######################################################################################################
#     ########  Classifier  ################################################################################
#     ######################################################################################################
    
#     y = tf.keras.layers.Dense(dense_decode_units, 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                               activation = 'relu')(input1)
    
#     y = tf.keras.layers.Dropout(prob)(y)
    
#     y = tf.keras.layers.Dense(int(dense_decode_units/2), 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                               activation = 'relu')(y)
    
#     y = tf.keras.layers.Dropout(prob)(y)
    
#     y = tf.keras.layers.Dense(1, 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                               activation = 'sigmoid')(y)

#     ## Generate Model from input and output
#     model = tf.keras.models.Model(inputs=input1, outputs=y)
    
#     ## Compile model
#     if(metrics != None):
#         model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
#                       loss = loss, metrics = metrics)
#     else:
#         model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
#                       loss = loss)

#     return model

In [10]:
# epochs = 100
# batch_size = 16

# ##################################################################################
# ##### Function to customize the DLNN architecture with parameters
# ##################################################################################

# def DLNN_Classifier(input_vec_shape,
#                     dense_decode_units = 32, ## Dense layer parameters
#                     prob = 0.5, learn_rate = 0.0005, loss = 'binary_crossentropy', metrics = 'accuracy'):
    
#     beta = 0.001
    
#     input1 = tf.keras.layers.Input(shape=input_vec_shape)
    
#     ######################################################################################################
#     ########  Classifier  ################################################################################
#     ######################################################################################################
    
#     y = tf.keras.layers.Dense(dense_decode_units, 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                               activation = 'relu')(input1)
    
#     y = tf.keras.layers.Dropout(prob)(y)
    
#     y = tf.keras.layers.Dense(int(dense_decode_units/2), 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                               activation = 'relu')(y)
    
#     y = tf.keras.layers.Dropout(prob)(y)
    
#     y = tf.keras.layers.Dense(1, 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                               activation = 'sigmoid')(y)

#     ## Generate Model from input and output
#     model = tf.keras.models.Model(inputs=input1, outputs=y)
    
#     ## Compile model
#     if(metrics != None):
#         model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
#                       loss = loss, metrics = metrics)
#     else:
#         model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
#                       loss = loss)

#     return model

In [11]:
# epochs = 100
# batch_size = 32

# ##################################################################################
# ##### Function to customize the DLNN architecture with parameters
# ##################################################################################

# def DLNN_Classifier(input_vec_shape,
#                     dense_decode_units = 50,
#                     dense_layers = 5,
#                     prob = 0.1, 
#                     learn_rate = 0.001, loss = 'binary_crossentropy', metrics = 'accuracy'):
    
#     beta = 0.0001
    
#     input1 = tf.keras.layers.Input(shape=input_vec_shape)
    
#     ######################################################################################################
#     ########  Classifier  ################################################################################
#     ######################################################################################################
    
#     y = tf.keras.layers.Dense(dense_decode_units, 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta)
#                              )(input1)
# #     y = tf.keras.layers.BatchNormalization()(y)
# #     y = tf.keras.layers.Dropout(prob)(y)
    
#     for i in range(1,dense_layers):
    
#         y = tf.keras.layers.Dense(int(dense_decode_units/(2**i)), 
#                                   kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                                  )(y)
# #         y = tf.keras.layers.BatchNormalization()(y)
# #         y = tf.keras.layers.Dropout(prob)(y)
    
#     y = tf.keras.layers.Dense(1, 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                               activation = 'sigmoid'
#                              )(y)

#     ## Generate Model from input and output
#     model = tf.keras.models.Model(inputs=input1, outputs=y)
    
#     ## Compile model
#     if(metrics != None):
#         model.compile(optimizer = tf.keras.optimizers.Adagrad(learning_rate=learn_rate), 
#                       loss = loss, metrics = metrics)
#     else:
#         model.compile(optimizer = tf.keras.optimizers.Adagrad(learning_rate=learn_rate), 
#                       loss = loss)

#     return model

In [12]:
DLNN_Classifier((400)).summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 400)]             0         
                                                                 
 dense (Dense)               (None, 32)                12832     
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 12,865
Trainable params: 12,865
Non-trainable params: 0
_________________________________________________________________


# Training

In [13]:
##################################################################################
##### For each input file, train model and generate different outputs in a structured folder
##################################################################################

## create the evaluation data structure for all iterations
evaluations = {
    "Fold" : [],
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

## Create and set directory to save model
modelPath = os.path.join(outPath, expName, "{}fold".format(n_fold), "models")
if(not os.path.isdir(modelPath)):
    os.makedirs(modelPath)

##################################################################################
##### TRAIN and PREDICT for every Fold, using models
##################################################################################

# fold counter
i = 0

for fold in folds:

    # adding random shuffling of the dataset for training purpose
    randomized_index_arr = np.arange(fold["X_train"].shape[0])
    randomized_index_arr = np.random.permutation(randomized_index_arr)

    print("\nTrain/Test model on Fold #"+str(i)+".")

    input_size = fold["X_train"][0].shape
    ## Generate model using function
    model = DLNN_Classifier(input_vec_shape = input_size)

    model_file_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    ## Define the model callbacks for early stopping and saving the model. Then train model
    modelCallbacks = [
        tf.keras.callbacks.ModelCheckpoint(model_file_path,
                                           monitor = monitor, verbose = 1, save_best_only = True, 
                                           save_weights_only = False, mode = 'auto', save_freq = 'epoch'),
    ]
    model.fit(x = fold["X_train"][randomized_index_arr], y = fold["y_train"][randomized_index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
              callbacks = modelCallbacks, validation_data = (fold["X_test"], fold["y_test"]))

    model = tf.keras.models.load_model(model_file_path)

    ##################################################################################
    ##### Prediction and metrics for TRAIN dataset
    ##################################################################################

    y_pred = model.predict(fold["X_train"])
    label_pred = pred2label(y_pred)
    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(fold["y_train"], label_pred)
    prec = precision_score(fold["y_train"],label_pred)

    mcc = matthews_corrcoef(fold["y_train"], label_pred)

    conf = confusion_matrix(fold["y_train"], label_pred)
    tn, fp, fn, tp = conf.ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)

    fpr, tpr, thresholds = roc_curve(fold["y_train"], y_pred)
    auc = roc_auc_score(fold["y_train"], y_pred)

    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Train")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)

    ##################################################################################
    ##### Prediction and metrics for TEST dataset
    ##################################################################################

    y_pred = model.predict(fold["X_test"])
    label_pred = pred2label(y_pred)
    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(fold["y_test"], label_pred)
    prec = precision_score(fold["y_test"],label_pred)
    mcc = matthews_corrcoef(fold["y_test"], label_pred)

    conf = confusion_matrix(fold["y_test"], label_pred)
    tn, fp, fn, tp = conf.ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)

    fpr, tpr, thresholds = roc_curve(fold["y_test"], y_pred)
    auc = roc_auc_score(fold["y_test"], y_pred)

    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Test")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)

    i = i+1
    del model
    tf.keras.backend.clear_session()


Train/Test model on Fold #0.
Epoch 1/100
116/120 [============================>.] - ETA: 0s - loss: 1.0164 - accuracy: 0.5447
Epoch 1: val_accuracy improved from -inf to 0.59119, saving model to Results\NT_Site_PredNTS_Classification_DLNN_AsimEmbedding_DDE\5fold\models\bestModel-fold0.hdf5
120/120 [==============================] - 1s 5ms/step - loss: 1.0153 - accuracy: 0.5449 - val_loss: 0.7670 - val_accuracy: 0.5912
Epoch 2/100
117/120 [============================>.] - ETA: 0s - loss: 0.7921 - accuracy: 0.6346
Epoch 2: val_accuracy improved from 0.59119 to 0.66247, saving model to Results\NT_Site_PredNTS_Classification_DLNN_AsimEmbedding_DDE\5fold\models\bestModel-fold0.hdf5
120/120 [==============================] - 0s 4ms/step - loss: 0.7940 - accuracy: 0.6341 - val_loss: 0.6940 - val_accuracy: 0.6625
Epoch 3/100
118/120 [============================>.] - ETA: 0s - loss: 0.6742 - accuracy: 0.6944
Epoch 3: val_accuracy improved from 0.66247 to 0.70860, saving model to Results\NT_S

Epoch 25/100
117/120 [============================>.] - ETA: 0s - loss: 0.2285 - accuracy: 0.9423
Epoch 25: val_accuracy improved from 0.76939 to 0.77568, saving model to Results\NT_Site_PredNTS_Classification_DLNN_AsimEmbedding_DDE\5fold\models\bestModel-fold0.hdf5
120/120 [==============================] - 0s 4ms/step - loss: 0.2284 - accuracy: 0.9428 - val_loss: 0.6037 - val_accuracy: 0.7757
Epoch 26/100
117/120 [============================>.] - ETA: 0s - loss: 0.2474 - accuracy: 0.9263
Epoch 26: val_accuracy improved from 0.77568 to 0.77778, saving model to Results\NT_Site_PredNTS_Classification_DLNN_AsimEmbedding_DDE\5fold\models\bestModel-fold0.hdf5
120/120 [==============================] - 1s 4ms/step - loss: 0.2474 - accuracy: 0.9265 - val_loss: 0.6173 - val_accuracy: 0.7778
Epoch 27/100
116/120 [============================>.] - ETA: 0s - loss: 0.2411 - accuracy: 0.9359
Epoch 27: val_accuracy did not improve from 0.77778
120/120 [==============================] - 1s 4ms/step

Epoch 53/100
117/120 [============================>.] - ETA: 0s - loss: 0.1477 - accuracy: 0.9749
Epoch 53: val_accuracy did not improve from 0.78407
120/120 [==============================] - 1s 4ms/step - loss: 0.1480 - accuracy: 0.9748 - val_loss: 0.7663 - val_accuracy: 0.7757
Epoch 54/100
115/120 [===========================>..] - ETA: 0s - loss: 0.1561 - accuracy: 0.9658
Epoch 54: val_accuracy did not improve from 0.78407
120/120 [==============================] - 1s 4ms/step - loss: 0.1583 - accuracy: 0.9643 - val_loss: 0.7709 - val_accuracy: 0.7799
Epoch 55/100
107/120 [=========================>....] - ETA: 0s - loss: 0.1579 - accuracy: 0.9685
Epoch 55: val_accuracy did not improve from 0.78407
120/120 [==============================] - 0s 4ms/step - loss: 0.1562 - accuracy: 0.9690 - val_loss: 0.7654 - val_accuracy: 0.7820
Epoch 56/100
118/120 [============================>.] - ETA: 0s - loss: 0.1560 - accuracy: 0.9682
Epoch 56: val_accuracy did not improve from 0.78407
120/120

118/120 [============================>.] - ETA: 0s - loss: 0.1376 - accuracy: 0.9762
Epoch 82: val_accuracy did not improve from 0.78407
120/120 [==============================] - 1s 4ms/step - loss: 0.1376 - accuracy: 0.9764 - val_loss: 0.8698 - val_accuracy: 0.7652
Epoch 83/100
118/120 [============================>.] - ETA: 0s - loss: 0.1292 - accuracy: 0.9793
Epoch 83: val_accuracy did not improve from 0.78407
120/120 [==============================] - 0s 4ms/step - loss: 0.1292 - accuracy: 0.9795 - val_loss: 0.8779 - val_accuracy: 0.7589
Epoch 84/100
120/120 [==============================] - ETA: 0s - loss: 0.1310 - accuracy: 0.9785
Epoch 84: val_accuracy did not improve from 0.78407
120/120 [==============================] - 0s 4ms/step - loss: 0.1310 - accuracy: 0.9785 - val_loss: 0.8746 - val_accuracy: 0.7568
Epoch 85/100
119/120 [============================>.] - ETA: 0s - loss: 0.1284 - accuracy: 0.9827
Epoch 85: val_accuracy did not improve from 0.78407
120/120 [===========

115/120 [===========================>..] - ETA: 0s - loss: 0.4692 - accuracy: 0.8087
Epoch 8: val_accuracy did not improve from 0.72327
120/120 [==============================] - 0s 4ms/step - loss: 0.4700 - accuracy: 0.8094 - val_loss: 0.5830 - val_accuracy: 0.7233
Epoch 9/100
114/120 [===========================>..] - ETA: 0s - loss: 0.4385 - accuracy: 0.8268
Epoch 9: val_accuracy improved from 0.72327 to 0.72956, saving model to Results\NT_Site_PredNTS_Classification_DLNN_AsimEmbedding_DDE\5fold\models\bestModel-fold1.hdf5
120/120 [==============================] - 0s 4ms/step - loss: 0.4407 - accuracy: 0.8262 - val_loss: 0.5762 - val_accuracy: 0.7296
Epoch 10/100
118/120 [============================>.] - ETA: 0s - loss: 0.4319 - accuracy: 0.8369
Epoch 10: val_accuracy did not improve from 0.72956
120/120 [==============================] - 0s 4ms/step - loss: 0.4307 - accuracy: 0.8373 - val_loss: 0.5745 - val_accuracy: 0.7296
Epoch 11/100
110/120 [==========================>...] - 

...

111/120 [==========================>...] - ETA: 0s - loss: 0.1227 - accuracy: 0.9809
Epoch 81: val_accuracy did not improve from 0.77101
120/120 [==============================] - 0s 4ms/step - loss: 0.1229 - accuracy: 0.9806 - val_loss: 1.0301 - val_accuracy: 0.7542
Epoch 82/100
116/120 [============================>.] - ETA: 0s - loss: 0.1273 - accuracy: 0.9795
Epoch 82: val_accuracy did not improve from 0.77101
120/120 [==============================] - 1s 4ms/step - loss: 0.1278 - accuracy: 0.9795 - val_loss: 1.0322 - val_accuracy: 0.7437
Epoch 83/100
107/120 [=========================>....] - ETA: 0s - loss: 0.1172 - accuracy: 0.9866
Epoch 83: val_accuracy did not improve from 0.77101
120/120 [==============================] - 0s 4ms/step - loss: 0.1204 - accuracy: 0.9843 - val_loss: 1.0512 - val_accuracy: 0.7584
Epoch 84/100
113/120 [===========================>..] - ETA: 0s - loss: 0.1245 - accuracy: 0.9773
Epoch 84: val_accuracy did not improve from 0.77101
120/120 [===========

120/120 [==============================] - ETA: 0s - loss: 0.4508 - accuracy: 0.8075
Epoch 8: val_accuracy improved from 0.71008 to 0.71849, saving model to Results\NT_Site_PredNTS_Classification_DLNN_AsimEmbedding_DDE\5fold\models\bestModel-fold4.hdf5
120/120 [==============================] - 1s 4ms/step - loss: 0.4508 - accuracy: 0.8075 - val_loss: 0.5943 - val_accuracy: 0.7185
Epoch 9/100
115/120 [===========================>..] - ETA: 0s - loss: 0.4300 - accuracy: 0.8255
Epoch 9: val_accuracy improved from 0.71849 to 0.73109, saving model to Results\NT_Site_PredNTS_Classification_DLNN_AsimEmbedding_DDE\5fold\models\bestModel-fold4.hdf5
120/120 [==============================] - 0s 4ms/step - loss: 0.4274 - accuracy: 0.8284 - val_loss: 0.5940 - val_accuracy: 0.7311
Epoch 10/100
113/120 [===========================>..] - ETA: 0s - loss: 0.4018 - accuracy: 0.8507
Epoch 10: val_accuracy improved from 0.73109 to 0.73739, saving model to Results\NT_Site_PredNTS_Classification_DLNN_AsimE

Epoch 34/100
114/120 [===========================>..] - ETA: 0s - loss: 0.1887 - accuracy: 0.9605
Epoch 34: val_accuracy did not improve from 0.75840
120/120 [==============================] - 1s 4ms/step - loss: 0.1887 - accuracy: 0.9607 - val_loss: 0.7386 - val_accuracy: 0.7437
Epoch 35/100
107/120 [=========================>....] - ETA: 0s - loss: 0.1814 - accuracy: 0.9667
Epoch 35: val_accuracy did not improve from 0.75840
120/120 [==============================] - 0s 4ms/step - loss: 0.1801 - accuracy: 0.9675 - val_loss: 0.7451 - val_accuracy: 0.7500
Epoch 36/100
118/120 [============================>.] - ETA: 0s - loss: 0.1745 - accuracy: 0.9640
Epoch 36: val_accuracy did not improve from 0.75840
120/120 [==============================] - 0s 4ms/step - loss: 0.1745 - accuracy: 0.9638 - val_loss: 0.7510 - val_accuracy: 0.7521
Epoch 37/100
116/120 [============================>.] - ETA: 0s - loss: 0.1768 - accuracy: 0.9553
Epoch 37: val_accuracy did not improve from 0.75840
120/120

Epoch 63/100
103/120 [========================>.....] - ETA: 0s - loss: 0.1362 - accuracy: 0.9769
Epoch 63: val_accuracy did not improve from 0.76050
120/120 [==============================] - 0s 4ms/step - loss: 0.1354 - accuracy: 0.9764 - val_loss: 0.8971 - val_accuracy: 0.7500
Epoch 64/100
117/120 [============================>.] - ETA: 0s - loss: 0.1335 - accuracy: 0.9744
Epoch 64: val_accuracy did not improve from 0.76050
120/120 [==============================] - 0s 4ms/step - loss: 0.1336 - accuracy: 0.9743 - val_loss: 0.8954 - val_accuracy: 0.7479
Epoch 65/100
114/120 [===========================>..] - ETA: 0s - loss: 0.1396 - accuracy: 0.9753
Epoch 65: val_accuracy did not improve from 0.76050
120/120 [==============================] - 0s 4ms/step - loss: 0.1397 - accuracy: 0.9753 - val_loss: 0.9168 - val_accuracy: 0.7521
Epoch 66/100
112/120 [===========================>..] - ETA: 0s - loss: 0.1339 - accuracy: 0.9794
Epoch 66: val_accuracy did not improve from 0.76050
120/120

Epoch 92/100
117/120 [============================>.] - ETA: 0s - loss: 0.1202 - accuracy: 0.9776
Epoch 92: val_accuracy did not improve from 0.76471
120/120 [==============================] - 0s 4ms/step - loss: 0.1208 - accuracy: 0.9774 - val_loss: 0.9701 - val_accuracy: 0.7542
Epoch 93/100
116/120 [============================>.] - ETA: 0s - loss: 0.1282 - accuracy: 0.9752
Epoch 93: val_accuracy did not improve from 0.76471
120/120 [==============================] - 0s 4ms/step - loss: 0.1282 - accuracy: 0.9759 - val_loss: 0.9674 - val_accuracy: 0.7500
Epoch 94/100
117/120 [============================>.] - ETA: 0s - loss: 0.1197 - accuracy: 0.9802
Epoch 94: val_accuracy did not improve from 0.76471
120/120 [==============================] - 0s 4ms/step - loss: 0.1194 - accuracy: 0.9801 - val_loss: 0.9815 - val_accuracy: 0.7605
Epoch 95/100
116/120 [============================>.] - ETA: 0s - loss: 0.1228 - accuracy: 0.9774
Epoch 95: val_accuracy did not improve from 0.76471
120/120

## k-fold Training evaluation

In [14]:
evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Test,0.779592,0.793673,0.842127,0.757319,0.801839,0.560414
Train,0.995802,0.997683,0.999884,0.993912,0.997691,0.991611


In [15]:
# 	Accuracy	Precision	AUC	Sensitivity	Specificity	MCC
# Train_Test						
# Test	0.784635	0.789820	0.844583	0.776646	0.792651	0.569722
# Train	0.975752	0.981026	0.994496	0.969969	0.981532	0.951614

In [16]:
evaluations_df

,Fold,Train_Test,Accuracy,Precision,TPR,FPR,TPR_FPR_Thresholds,AUC,Sensitivity,Specificity,MCC
0,0,Train,0.991601,0.992632,"[0.0, 0.0021008403361344537, 0.031512605042016...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.9999996, 0.99999964, 0.99987566, 0.99986005...",0.999599,0.990546,0.992655,0.983204
1,0,Test,0.784067,0.780992,"[0.0, 0.0041841004184100415, 0.046025104602510...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.9999994, 0.9999994, 0.99960035, 0.9995989, ...",0.849135,0.790795,0.777311,0.568166
2,1,Train,0.997900,0.998948,"[0.0, 0.014690451206715634, 0.0199370409233997...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2.0, 1.0, 0.9999999, 0.99999976, 0.99999964, ...",0.999981,0.996852,0.998950,0.995803
3,1,Test,0.786164,0.800885,"[0.0, 0.008403361344537815, 0.0168067226890756...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.004...","[2.0, 1.0, 0.9999999, 0.99999964, 0.9999995, 0...",0.852009,0.760504,0.811715,0.573006
4,2,Train,0.996327,0.998945,"[0.0, 0.0062959076600209865, 0.014690451206715...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2.0, 1.0, 0.9999999, 0.99999976, 0.99999964, ...",0.999917,0.993704,0.998951,0.992668
5,2,Test,0.792017,0.835749,"[0.0, 0.01680672268907563, 0.02521008403361344...","[0.0, 0.0, 0.0, 0.0, 0.004201680672268907, 0.0...","[2.0, 1.0, 0.99999976, 0.99990416, 0.9998877, ...",0.852412,0.726891,0.857143,0.589052
6,3,Train,0.996327,0.998945,"[0.0, 0.01888772298006296, 0.02833158447009444...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2.0, 1.0, 0.9999999, 0.99999976, 0.9999995, 0...",0.999947,0.993704,0.998951,0.992668
7,3,Test,0.771008,0.776824,"[0.0, 0.01680672268907563, 0.02521008403361344...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00420168...","[2.0, 1.0, 0.9999999, 0.999997, 0.9999964, 0.9...",0.819231,0.760504,0.781513,0.542136
8,4,Train,0.996852,0.998946,"[0.0, 0.016789087093389297, 0.0272822665267576...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2.0, 1.0, 0.9999999, 0.99999976, 0.99999964, ...",0.999974,0.994753,0.998951,0.993713
9,4,Test,0.764706,0.773913,"[0.0, 0.02100840336134454, 0.02941176470588235...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00420168...","[2.0, 1.0, 0.99999964, 0.9999995, 0.99999917, ...",0.837847,0.747899,0.781513,0.529711


# Independent data

In [17]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Independent Data performance
##################################################################################

print("\nIndependent evaluation for model.")

# adding random shuffling of the dataset for training purpose
randomized_index_arr = np.arange(train_features.shape[0])
randomized_index_arr = np.random.permutation(randomized_index_arr)

input_size = train_features[0].shape
## Generate model using function
model = DLNN_Classifier(input_vec_shape = input_size)

model_file_path = os.path.join(modelPath, "bestModel-full.hdf5")
## Define the model callbacks for early stopping and saving the model. Then train model
modelCallbacks = [
    tf.keras.callbacks.ModelCheckpoint(model_file_path,
                                       monitor = monitor, verbose = 1, save_best_only = True, 
                                       save_weights_only = False, mode = 'auto', save_freq = 'epoch'),
]
model.fit(x = train_features[randomized_index_arr], y = train_labels[randomized_index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
          callbacks = modelCallbacks, validation_data = (indpe_features, indpe_labels))

model = tf.keras.models.load_model(model_file_path)

##################################################################################
##### Prediction and metrics for Train dataset
##################################################################################

y_pred = model.predict(train_features)
label_pred = pred2label(y_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(train_labels, label_pred)
prec = precision_score(train_labels,label_pred)
mcc = matthews_corrcoef(train_labels, label_pred)

conf = confusion_matrix(train_labels, label_pred)
tn, fp, fn, tp = conf.ravel()
sens = tp/(tp+fn)
spec = tn/(tn+fp)

fpr, tpr, thresholds = roc_curve(train_labels, y_pred)
auc = roc_auc_score(train_labels, y_pred)

evaluations["Train_Test"].append("Train")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

y_pred = model.predict(indpe_features)
label_pred = pred2label(y_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)
mcc = matthews_corrcoef(indpe_labels, label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
tn, fp, fn, tp = conf.ravel()
sens = tp/(tp+fn)
spec = tn/(tn+fp)

fpr, tpr, thresholds = roc_curve(indpe_labels, y_pred)
auc = roc_auc_score(indpe_labels, y_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

del model
tf.keras.backend.clear_session()


Independent evaluation for model.
Epoch 1/100
148/149 [============================>.] - ETA: 0s - loss: 0.9970 - accuracy: 0.5536
Epoch 1: val_accuracy improved from -inf to 0.52163, saving model to Results\NT_Site_PredNTS_Classification_DLNN_AsimEmbedding_DDE\5fold\models\bestModel-full.hdf5
149/149 [==============================] - 1s 6ms/step - loss: 0.9989 - accuracy: 0.5533 - val_loss: 0.9046 - val_accuracy: 0.5216
Epoch 2/100
138/149 [==========================>...] - ETA: 0s - loss: 0.7751 - accuracy: 0.6304
Epoch 2: val_accuracy improved from 0.52163 to 0.58367, saving model to Results\NT_Site_PredNTS_Classification_DLNN_AsimEmbedding_DDE\5fold\models\bestModel-full.hdf5
149/149 [==============================] - 1s 4ms/step - loss: 0.7639 - accuracy: 0.6352 - val_loss: 0.8170 - val_accuracy: 0.5837
Epoch 3/100
134/149 [=========================>....] - ETA: 0s - loss: 0.6485 - accuracy: 0.6964
Epoch 3: val_accuracy did not improve from 0.58367
149/149 [=====================

139/149 [==========================>...] - ETA: 0s - loss: 0.2349 - accuracy: 0.9397
Epoch 27: val_accuracy did not improve from 0.61306
149/149 [==============================] - 1s 4ms/step - loss: 0.2340 - accuracy: 0.9416 - val_loss: 1.2042 - val_accuracy: 0.6090
Epoch 28/100
148/149 [============================>.] - ETA: 0s - loss: 0.2264 - accuracy: 0.9455
Epoch 28: val_accuracy did not improve from 0.61306
149/149 [==============================] - 1s 4ms/step - loss: 0.2270 - accuracy: 0.9454 - val_loss: 1.2357 - val_accuracy: 0.6098
Epoch 29/100
147/149 [============================>.] - ETA: 0s - loss: 0.2241 - accuracy: 0.9418
Epoch 29: val_accuracy did not improve from 0.61306
149/149 [==============================] - 1s 4ms/step - loss: 0.2240 - accuracy: 0.9416 - val_loss: 1.2616 - val_accuracy: 0.6106
Epoch 30/100
148/149 [============================>.] - ETA: 0s - loss: 0.2148 - accuracy: 0.9472
Epoch 30: val_accuracy improved from 0.61306 to 0.61388, saving model to

149/149 [==============================] - 1s 5ms/step - loss: 0.1559 - accuracy: 0.9689 - val_loss: 1.7593 - val_accuracy: 0.5984
Epoch 56/100
142/149 [===========================>..] - ETA: 0s - loss: 0.1587 - accuracy: 0.9652
Epoch 56: val_accuracy did not improve from 0.61633
149/149 [==============================] - 1s 5ms/step - loss: 0.1581 - accuracy: 0.9664 - val_loss: 1.7148 - val_accuracy: 0.6098
Epoch 57/100
142/149 [===========================>..] - ETA: 0s - loss: 0.1536 - accuracy: 0.9718
Epoch 57: val_accuracy did not improve from 0.61633
149/149 [==============================] - 1s 4ms/step - loss: 0.1523 - accuracy: 0.9723 - val_loss: 1.7924 - val_accuracy: 0.6090
Epoch 58/100
135/149 [==========================>...] - ETA: 0s - loss: 0.1544 - accuracy: 0.9676
Epoch 58: val_accuracy did not improve from 0.61633
149/149 [==============================] - 1s 4ms/step - loss: 0.1542 - accuracy: 0.9685 - val_loss: 1.7493 - val_accuracy: 0.6114
Epoch 59/100
133/149 [====

137/149 [==========================>...] - ETA: 0s - loss: 0.1446 - accuracy: 0.9731
Epoch 84: val_accuracy did not improve from 0.61959
149/149 [==============================] - 1s 5ms/step - loss: 0.1440 - accuracy: 0.9740 - val_loss: 2.0320 - val_accuracy: 0.6090
Epoch 85/100
136/149 [==========================>...] - ETA: 0s - loss: 0.1276 - accuracy: 0.9816
Epoch 85: val_accuracy did not improve from 0.61959
149/149 [==============================] - 1s 5ms/step - loss: 0.1309 - accuracy: 0.9803 - val_loss: 2.0645 - val_accuracy: 0.6057
Epoch 86/100
147/149 [============================>.] - ETA: 0s - loss: 0.1424 - accuracy: 0.9698
Epoch 86: val_accuracy did not improve from 0.61959
149/149 [==============================] - 1s 4ms/step - loss: 0.1424 - accuracy: 0.9702 - val_loss: 2.0360 - val_accuracy: 0.6114
Epoch 87/100
138/149 [==========================>...] - ETA: 0s - loss: 0.1381 - accuracy: 0.9769
Epoch 87: val_accuracy did not improve from 0.61959
149/149 [===========

In [18]:
evaluations_df = pd.DataFrame.from_dict(evaluations)
evaluations_df

,Train_Test,Accuracy,Precision,TPR,FPR,TPR_FPR_Thresholds,AUC,Sensitivity,Specificity,MCC
0,Train,0.995802,0.997473,"[0.0, 0.02938706968933669, 0.04198152812762384...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2.0, 1.0, 0.9999999, 0.99999976, 0.99999964, ...",0.999929,0.994123,0.997481,0.991609
1,Independent,0.619592,0.240631,"[0.0, 0.034482758620689655, 0.0492610837438423...","[0.0, 0.007827788649706457, 0.0136986301369863...","[2.0, 1.0, 0.9999999, 0.99999976, 0.99999964, ...",0.651369,0.600985,0.623288,0.169311


In [19]:
# 	Train_Test	Accuracy	Precision	TPR	FPR	TPR_FPR_Thresholds	AUC	Sensitivity	Specificity	MCC
# 0	Train	0.995382	0.996633	[0.0, 0.008396305625524769, 0.0134340890008396...	[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...	[2.0, 1.0, 0.9999999, 0.99999976, 0.99999964, ...	0.999800	0.994123	0.996641	0.990767
# 1	Independent	0.644898	0.247826	[0.0, 0.014778325123152709, 0.0197044334975369...	[0.0, 0.0009784735812133072, 0.002935420743639...	[2.0, 1.0, 0.9999999, 0.99999976, 0.99999917, ...	0.647742	0.561576	0.661448	0.171244